# AI/ML i GenAI Integrations

**Cel szkoleniowy:** Zrozumienie mozliwosci AI/ML w Databricks: MLflow, Feature Store, Vector Search i GenAI

**Zakres tematyczny:**
- MLflow: Tracking, Registry, Model Serving
- Feature Store: centralne repozytorium cech
- Vector Search: baza wektorowa dla RAG
- GenAI: integracja z LLM (DBRX, Llama)


## Setup i konfiguracja

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 3 - Integracje i Governance
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
 - Klaster ML (Databricks Runtime ML 13.0+)
 - Dostęp do Unity Catalog (dla Feature Store i Vector Search)
 - (Opcjonalnie) Skonfigurowany Vector Search Endpoint
 - (Opcjonalnie) Dostęp do Model Serving (Foundation Models)

## Wstęp teoretyczny

**Cel sekcji:** Zrozumienie jak Databricks wspiera pełny cykl życia AI/ML.

**Podstawowe pojęcia:**
- **MLflow**: Platforma open-source do zarządzania cyklem życia ML (eksperymenty, modele, wdrożenia).
- **Feature Engineering (Feature Store)**: Scentralizowane repozytorium cech w Unity Catalog, zapewniające spójność między treningiem a inferencją.
- **Vector Search**: Baza danych wektorowych zintegrowana z Delta Lake, kluczowa dla RAG (Retrieval Augmented Generation).
- **Model Serving**: Wdrażanie modeli jako endpointy REST API (w tym Foundation Models jak DBRX, Llama).

## Izolacja per użytkownik

In [0]:
%run ../00_setup

## Konfiguracja środowiska

In [0]:
import mlflow
import pandas as pd
from pyspark.sql import functions as F
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Feature Engineering Client (Unity Catalog - nowy sposób)
from databricks.feature_engineering import FeatureEngineeringClient

# Ustawienie katalogu i schematu
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {GOLD_SCHEMA}")

display(spark.createDataFrame([
    ("Katalog", CATALOG),
    ("Schemat", GOLD_SCHEMA)
], ["Parametr", "Wartość"]))

## Sekcja 1: MLflow - Tracking & Registry

Zbudujemy prosty model przewidujący wartość zamówienia i zarejestrujemy go w MLflow.

In [0]:
# 1. Konfiguracja eksperymentu
username = spark.sql("SELECT current_user()").collect()[0][0]
experiment_path = f"/Users/{username}/KION_ML_Experiment"

mlflow.set_experiment(experiment_path)

display(spark.createDataFrame([("Eksperyment MLflow", experiment_path)], ["Parametr", "Wartość"]))

In [0]:
%python
# 2. Przygotowanie danych
df_sales = spark.sql(
    f"""
    SELECT 
        net_amount as label,
        quantity,
        month(order_ts) as month,
        year(order_ts) as year
    FROM {CATALOG}.{GOLD_SCHEMA}.fact_sales
    WHERE net_amount IS NOT NULL
    """
).toPandas()

X = df_sales.drop("label", axis=1)
y = df_sales["label"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

display(
    spark.createDataFrame(
        [
            ("Train shape", f"{X_train.shape[0]} rows, {X_train.shape[1]} features"),
            ("Test shape", f"{X_test.shape[0]} rows, {X_test.shape[1]} features")
        ],
        ["Dataset", "Rozmiar"]
    )
)

In [0]:
# 3. Trening i logowanie
with mlflow.start_run(run_name="RandomForest_v1") as run:
    # Parametry
    n_estimators = 50
    mlflow.log_param("n_estimators", n_estimators)
    
    # Model
    model = RandomForestRegressor(n_estimators=n_estimators)
    model.fit(X_train, y_train)
    
    # Metryki
    mse = mean_squared_error(y_test, model.predict(X_test))
    mlflow.log_metric("mse", mse)
    
    # Logowanie modelu
    mlflow.sklearn.log_model(model, "model")
    
    run_id = run.info.run_id

display(spark.createDataFrame([
    ("Run ID", run_id),
    ("MSE", f"{mse:.2f}"),
    ("Model", "RandomForestRegressor"),
    ("n_estimators", str(n_estimators))
], ["Parametr", "Wartość"]))

## Sekcja 2: Feature Store

Feature Store pozwala na definiowanie cech raz i używanie ich wszędzie.

In [0]:
from databricks.feature_store import FeatureStoreClient

In [0]:
# Definicja tabeli cech (Feature Table)
fs = FeatureStoreClient()
feature_table_name = f"{CATALOG}.{GOLD_SCHEMA}.customer_features_{username.split('@')[0].replace('.', '_')}"

# Logika obliczania cech
features_df = spark.sql(
    f"""
    SELECT 
        customer_id,
        count(*) as order_count,
        sum(net_amount) as total_spend,
        avg(net_amount) as avg_order_value
    FROM {CATALOG}.{GOLD_SCHEMA}.fact_sales
    GROUP BY customer_id
    """
)

# Zapis do Feature Store
# Uwaga: create_table wymaga unikalnej nazwy i Primary Key

fs.create_table(
    name=feature_table_name,
    primary_keys=["customer_id"],
    df=features_df,
    description="Podstawowe cechy klienta: liczba zamówień, wydatki."
)
status_msg = "Utworzona"


display(spark.createDataFrame([
    ("Feature Table", feature_table_name),
    ("Status", status_msg),
    ("Primary Key", "customer_id"),
    ("Cechy", "order_count, total_spend, avg_order_value")
], ["Parametr", "Wartość"]))

## Sekcja 3: Vector Search (RAG Foundation)

Vector Search to klucz do budowania systemów RAG. Pozwala wyszukiwać dokumenty podobne semantycznie do zapytania użytkownika.

**Wymagania:**
1. **Vector Search Endpoint** (musi być utworzony w Compute -> Vector Search).
2. **Tabela źródłowa** z włączonym Change Data Feed.

In [0]:
# 1. Przygotowanie danych tekstowych (Symulacja bazy wiedzy)
# Tworzymy tabelę z opisami produktów (w realnym scenariuszu to byłyby PDFy, dokumentacja itp.)

docs_df = spark.createDataFrame([
    (1, "Wózek widłowy elektryczny, udźwig 2 tony, do pracy wewnątrz magazynu."),
    (2, "Wózek spalinowy Diesel, udźwig 5 ton, do pracy na zewnątrz, trudny teren."),
    (3, "Ręczny wózek paletowy, udźwig 500kg, do lekkich prac."),
    (4, "System automatyzacji magazynu, regały wysokiego składowania.")
], ["id", "text"])

source_table = f"{CATALOG}.{GOLD_SCHEMA}.product_docs_rag"
docs_df.write.format("delta").mode("overwrite").option("delta.enableChangeDataFeed", "true").saveAsTable(source_table)

display(spark.createDataFrame([
    ("Tabela źródłowa", source_table),
    ("Liczba dokumentów", "4"),
    ("Change Data Feed", "Włączony")
], ["Parametr", "Wartość"]))

In [0]:

%skip 
%pip install databricks-vectorsearch
dbutils.library.restartPython()

In [0]:

# 2. Konfiguracja Vector Search (Kod właściwy)
# Uwaga: To wymaga aktywnego Endpointu Vector Search. 
# Jeśli go nie ma, kod rzuci błąd, ale pokazuje poprawną ścieżkę implementacji.

from databricks.vector_search.client import VectorSearchClient

# Nazwa endpointu (musi istnieć w UI)
VECTOR_SEARCH_ENDPOINT_NAME = "product_docs_index"
index_name = f"{CATALOG}.{GOLD_SCHEMA}.product_docs_idx"

status_info = []

try:
    vsc = VectorSearchClient()
    # Tworzenie indeksu (Delta Sync Index)
    vsc.create_delta_sync_index(
        endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
        source_table_name=source_table,
        index_name=index_name,
        pipeline_type="TRIGGERED",
        primary_key="id",
        embedding_source_column="text",
        embedding_model_endpoint_name="databricks-gte-large-en"
    )
    status_info.append(("Status", "Indeks utworzony"))
    status_info.append(("Endpoint", VECTOR_SEARCH_ENDPOINT_NAME))
    status_info.append(("Index Name", index_name))
except Exception as e:
    status_info.append(("Status", f"Błąd: {e}"))

display(spark.createDataFrame(status_info, ["Parametr", "Wartość"]))

In [0]:

# 3. Symulacja wyszukiwania (gdyby Vector Search działał)

query = "Szukam wózka do ciężkich prac na dworze"
display(spark.sql(f"""
SELECT
    *
FROM
    VECTOR_SEARCH(
        index => '{index_name}',
        query_text => '{query}',
        num_results => 3
    )
"""))

In [0]:

# --- SYMULACJA LOKALNA (Dla celów edukacyjnych, gdy brak infrastruktury) ---
display(spark.createDataFrame([
    ("Query", "Szukam wózka do ciężkich prac na dworze"),
    ("", ""),
    ("Wynik - ID", "2"),
    ("Wynik - Text", "Wózek spalinowy Diesel, udźwig 5 ton, do pracy na zewnątrz, trudny teren."),
    ("Wynik - Score", "0.89")
], ["Parametr", "Wartość"]))

## Sekcja 4: GenAI & Model Serving

Integracja z modelami LLM (Large Language Models) poprzez Databricks Model Serving.

In [0]:
# Wywołanie modelu LLM (np. DBRX, Llama 3)
import mlflow.deployments

# Lista dostępnych endpointów (może wymagać uprawnień)
# client = mlflow.deployments.get_deploy_client("databricks")
# endpoints = client.list_endpoints()

def query_llm(prompt):
    try:
        client = mlflow.deployments.get_deploy_client("databricks")
        
        # Używamy modelu 'databricks-claude-sonnet-4-5' lub 'databricks-meta-llama-3-70b-instruct'
        # Te modele są często dostępne jako "Pay-per-token"
        response = client.predict(
            endpoint="databricks-claude-sonnet-4-5",
            inputs={
                "messages": [
                    {"role": "system", "content": "Jesteś ekspertem od wózków widłowych."},
                    {"role": "user", "content": prompt}
                ],
                "max_tokens": 100
            }
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        return f"Nie udało się połączyć z modelem: {e}"

# Test
prompt = "Jakie są zalety wózków elektrycznych?"
odpowiedz = query_llm(prompt)

display(spark.createDataFrame([
    ("Pytanie", prompt),
    ("Odpowiedź LLM", odpowiedz)
], ["Parametr", "Wartość"]))

## Sekcja 5: AI Functions - Wbudowane funkcje AI w SQL

**Databricks AI Functions** to natywne funkcje SQL, które pozwalają na wykorzystanie LLM bezpośrednio w zapytaniach SQL. 
Są dostępne w Unity Catalog i działają na poziomie kolumn.

**Dostępne funkcje:**

| Funkcja | Opis | Przypadek użycia |
|---------|------|------------------|
| `ai_query()` | Wywołanie dowolnego prompta LLM | Generowanie, tłumaczenie, streszczenie |
| `ai_analyze_sentiment()` | Analiza sentymentu tekstu | Opinie klientów, recenzje |
| `ai_classify()` | Klasyfikacja tekstu do kategorii | Routing ticketów, tagowanie |
| `ai_extract()` | Ekstrakcja strukturyzowanych danych | Parsowanie dokumentów |
| `ai_fix_grammar()` | Korekta gramatyczna | Czyszczenie danych tekstowych |
| `ai_gen()` | Generowanie tekstu | Opisy produktów, emaile |
| `ai_similarity()` | Podobieństwo semantyczne tekstów | Deduplikacja, matching |
| `ai_summarize()` | Streszczenie tekstu | Podsumowania artykułów |
| `ai_translate()` | Tłumaczenie tekstu | Internacjonalizacja |

> **Uwaga:** Te funkcje wymagają aktywnego Model Serving endpoint i są płatne per token.

### 5.1 ai_query() - Uniwersalna funkcja LLM

`ai_query()` to najpotężniejsza funkcja - pozwala wywołać dowolny prompt na kolumnach tabeli.

In [ ]:
-- Przykład 5.1a: Generowanie opisów produktów
-- Używamy ai_query() do wzbogacenia danych o opisy wygenerowane przez LLM

SELECT 
    product_id,
    product_name,
    category,
    price,
    ai_query(
        'databricks-meta-llama-3-3-70b-instruct',  -- nazwa modelu
        CONCAT(
            'Napisz krótki, atrakcyjny opis marketingowy (max 2 zdania) dla produktu: ',
            product_name, 
            ' z kategorii: ', category,
            ' w cenie: ', price, ' USD'
        )
    ) AS generated_description
FROM workshop.silver.products
LIMIT 3;

-- UWAGA: Funkcja ai_query() wymaga dostępnego endpointu Foundation Model
-- W środowisku bez endpointu otrzymasz błąd

In [ ]:
-- Przykład 5.1b: Tłumaczenie danych na inny język
SELECT 
    customer_id,
    first_name,
    city,
    state,
    ai_query(
        'databricks-meta-llama-3-3-70b-instruct',
        CONCAT('Przetłumacz na polski: Customer from ', city, ', ', state)
    ) AS polish_description
FROM workshop.silver.customers
LIMIT 3;

### 5.2 ai_analyze_sentiment() - Analiza sentymentu

Automatyczna analiza sentymentu tekstu (positive/negative/neutral).

In [ ]:
-- Przykład 5.2: Analiza sentymentu opinii klientów
-- Tworzymy tymczasowe dane z opiniami do analizy

WITH customer_reviews AS (
    SELECT 1 AS review_id, 'Świetny produkt! Bardzo polecam.' AS review_text
    UNION ALL
    SELECT 2, 'Okropna jakość, nigdy więcej nie kupię.'
    UNION ALL
    SELECT 3, 'Produkt jest OK, nic specjalnego.'
    UNION ALL
    SELECT 4, 'Fantastyczna obsługa klienta, super szybka dostawa!'
    UNION ALL
    SELECT 5, 'Rozczarowany. Produkt nie działa zgodnie z opisem.'
)
SELECT 
    review_id,
    review_text,
    ai_analyze_sentiment(review_text) AS sentiment
FROM customer_reviews;

-- Wynik: sentiment = 'positive', 'negative' lub 'neutral' dla każdej opinii

### 5.3 ai_similarity() - Podobieństwo semantyczne

Porównuje dwa teksty i zwraca wartość podobieństwa (0-1).

In [ ]:
-- Przykład 5.3: Znajdowanie podobnych produktów
-- Użyteczne do deduplikacji lub rekomendacji

WITH product_pairs AS (
    SELECT 
        'Laptop Dell Inspiron 15' AS product_a,
        'Notebook Dell Inspiron 15.6 cali' AS product_b
    UNION ALL
    SELECT 
        'iPhone 15 Pro Max 256GB',
        'Samsung Galaxy S24 Ultra'
    UNION ALL
    SELECT 
        'Kabel USB-C 1m',
        'Przewód USB Type-C 100cm'
)
SELECT 
    product_a,
    product_b,
    ai_similarity(product_a, product_b) AS similarity_score
FROM product_pairs
ORDER BY similarity_score DESC;

-- similarity_score > 0.8 = prawdopodobne duplikaty
-- similarity_score 0.5-0.8 = podobne produkty
-- similarity_score < 0.5 = różne produkty

### 5.4 ai_extract() - Ekstrakcja strukturyzowanych danych

Wyciąga ustrukturyzowane informacje z tekstu (dawniej ai_parse_document).

In [ ]:
-- Przykład 5.4: Ekstrakcja danych z nieustrukturyzowanego tekstu
-- Parsowanie informacji kontaktowych z wiadomości

WITH messages AS (
    SELECT 1 AS msg_id, 
           'Dzień dobry, nazywam się Jan Kowalski. Mój numer telefonu to 512-345-678, email: jan.kowalski@example.com' AS message
    UNION ALL
    SELECT 2, 
           'Proszę o kontakt: Anna Nowak, tel. 601 234 567, anna.nowak@firma.pl'
    UNION ALL
    SELECT 3, 
           'Zamówienie złożone przez Piotra Wiśniewskiego, kontakt: piotr.w@gmail.com, +48 700 800 900'
)
SELECT 
    msg_id,
    message,
    ai_extract(
        message,
        ARRAY('imie_nazwisko', 'telefon', 'email')
    ) AS extracted_data
FROM messages;

-- Wynik: STRUCT z polami imie_nazwisko, telefon, email dla każdej wiadomości

### 5.5 ai_classify() - Klasyfikacja tekstu

Przypisuje tekst do jednej z predefiniowanych kategorii.

In [ ]:
-- Przykład 5.5: Automatyczne kategoryzowanie ticketów supportowych

WITH support_tickets AS (
    SELECT 1 AS ticket_id, 'Nie mogę się zalogować do systemu' AS issue
    UNION ALL
    SELECT 2, 'Kiedy otrzymam zwrot pieniędzy za zamówienie #12345?'
    UNION ALL
    SELECT 3, 'Produkt przyszedł uszkodzony, zdjęcia w załączniku'
    UNION ALL
    SELECT 4, 'Jak zmienić hasło do konta?'
    UNION ALL
    SELECT 5, 'Chcę anulować subskrypcję premium'
)
SELECT 
    ticket_id,
    issue,
    ai_classify(
        issue,
        ARRAY('problemy_techniczne', 'zwroty_reklamacje', 'platnosci', 'konto_ustawienia', 'anulowanie')
    ) AS category
FROM support_tickets;

-- Automatyczny routing ticketów do odpowiednich zespołów!

### 5.6 ai_summarize() i ai_translate() - Streszczanie i tłumaczenie

In [ ]:
-- Przykład 5.6a: Streszczanie długich opisów produktów

WITH product_descriptions AS (
    SELECT 1 AS product_id, 
           'Laptop biznesowy Dell Latitude 5540 to wszechstronny notebook zaprojektowany z myślą o profesjonalistach. Wyposażony w najnowszy procesor Intel Core i7 13. generacji, 16GB pamięci RAM DDR5 oraz szybki dysk SSD NVMe 512GB. Matowy ekran IPS 15.6 cala o rozdzielczości Full HD zapewnia doskonałą czytelność nawet w jasnym świetle. Aluminiowa obudowa gwarantuje trwałość, a bateria zapewnia do 10 godzin pracy. Zintegrowane zabezpieczenia obejmują czytnik linii papilarnych, TPM 2.0 oraz kamerę IR do Windows Hello.' AS full_description
)
SELECT 
    product_id,
    ai_summarize(full_description) AS short_summary
FROM product_descriptions;

In [ ]:
-- Przykład 5.6b: Tłumaczenie opisów produktów na wiele języków

SELECT 
    product_id,
    product_name,
    ai_translate(product_name, 'de') AS german_name,
    ai_translate(product_name, 'es') AS spanish_name,
    ai_translate(product_name, 'fr') AS french_name
FROM workshop.silver.products
LIMIT 5;

-- Wspierane języki: 'en', 'de', 'es', 'fr', 'pl', 'it', 'pt', 'nl', 'ja', 'ko', 'zh' i więcej

### 5.7 Praktyczny przypadek użycia: Wzbogacenie danych klientów

Połączenie wielu AI Functions w jednym pipeline do analizy i wzbogacenia danych.

In [ ]:
-- Przykład 5.7: Kompletny pipeline analizy feedbacku klientów

WITH customer_feedback AS (
    SELECT 
        1 AS feedback_id,
        'John Smith' AS customer_name,
        'I absolutely love this product! Fast shipping and great quality. Will buy again!' AS feedback_text
    UNION ALL
    SELECT 2, 'Maria Garcia', 'Product arrived damaged. Customer service was unhelpful. Very disappointed.'
    UNION ALL
    SELECT 3, 'Klaus Mueller', 'Decent product for the price. Nothing special but works as expected.'
)
SELECT 
    feedback_id,
    customer_name,
    feedback_text,
    
    -- Analiza sentymentu
    ai_analyze_sentiment(feedback_text) AS sentiment,
    
    -- Klasyfikacja do kategorii
    ai_classify(
        feedback_text, 
        ARRAY('jakość_produktu', 'dostawa', 'obsługa_klienta', 'cena')
    ) AS main_topic,
    
    -- Streszczenie (dla długich tekstów)
    ai_summarize(feedback_text) AS summary,
    
    -- Tłumaczenie na polski
    ai_translate(feedback_text, 'pl') AS polish_translation
    
FROM customer_feedback;

-- Wynik: Kompletna analiza każdego feedbacku w jednym zapytaniu!

### 📌 Podsumowanie AI Functions

| Funkcja | Składnia | Przykład użycia |
|---------|----------|-----------------|
| `ai_query()` | `ai_query(model, prompt)` | Generowanie, dowolny prompt |
| `ai_analyze_sentiment()` | `ai_analyze_sentiment(text)` | Analiza opinii |
| `ai_classify()` | `ai_classify(text, categories)` | Routing ticketów |
| `ai_extract()` | `ai_extract(text, fields)` | Parsowanie dokumentów |
| `ai_similarity()` | `ai_similarity(text1, text2)` | Deduplikacja |
| `ai_summarize()` | `ai_summarize(text)` | Skracanie opisów |
| `ai_translate()` | `ai_translate(text, lang)` | Internacjonalizacja |
| `ai_fix_grammar()` | `ai_fix_grammar(text)` | Korekta tekstu |
| `ai_gen()` | `ai_gen(prompt)` | Generowanie treści |

**Wymagania:**
- Unity Catalog 
- Foundation Model APIs lub własny Model Serving endpoint
- Uprawnienia: `USE CATALOG`, `USE SCHEMA`, oraz dostęp do modelu

**Koszty:** Rozliczane per token - pamiętaj o optymalizacji promptów!

## Podsumowanie

1. Zbudowaliśmy pipeline treningowy w **MLflow**.
2. Utworzyliśmy tabelę cech w **Feature Store**.
3. Pokazaliśmy jak skonfigurować **Vector Search** dla RAG.
4. Skorzystaliśmy z **GenAI** do generowania odpowiedzi.

## Czyszczenie zasobów

In [0]:
# spark.sql(f"DROP TABLE IF EXISTS {feature_table_name}")
# spark.sql(f"DROP TABLE IF EXISTS {source_table}")
display(spark.createDataFrame([("Status", "Zasoby zachowane do dalszych ćwiczeń")], ["Info", "Wartość"]))